In [2]:
patient_image=[]
year_image=[]
weight_folders=[]

In [ ]:
import pandas as pd

csv_path = r'/home/omen/Documents/VS_data/vs_paths.csv'
df = pd.read_csv(csv_path)

In [ ]:
import os
data_dir = r'\home\omen\Documents\VS_data'

dcm_paths = []
weight_folders = list(df['image_path'])
weight_folders = [os.path.join(data_dir,p) for p in weight_folders]
weight_folders = [p.replace("\\", "/") for p in weight_folders]

for i in range(len(weight_folders)):
    dcm_files = [f for f in os.listdir(weight_folders[i]) if f.endswith('.dcm')]
    dcm_paths.append(os.path.join(weight_folders[i], dcm_files[0]))

In [ ]:
seg_paths = list(df['SegmentationPath'])
seg_paths = [os.path.join(data_dir,p) for p in seg_paths]
seg_paths = [p.replace("\\", "/") for p in seg_paths]

In [ ]:
import re

def extract_patient_name(path):
    pattern = r"/(?P<patient_name>(?:VS-(?:SEG|MC-RC)-)?\d+)/(?P<date_of_mri>\d{2}-\d{2}-\d{4})"
    match = re.search(pattern, path)
    if match:
        return match.group(1)
    return None

def extract_patient_year(path):
  pattern = r"/(?P<patient_name>(?:VS-(?:SEG|MC-RC)-)?\d+)/(?P<date_of_mri>\d{2}-\d{2}-\d{4})"
  match = re.search(pattern, path)
  if match:
    return match.group(2)
  return None

In [ ]:
import SimpleITK as sitk
import numpy as np
import matplotlib.pyplot as plt

def resample_segmentation_to_image(segmentation_image, image):
    resampler = sitk.ResampleImageFilter()
    resampler.SetSize(image.GetSize())
    resampler.SetOutputSpacing(image.GetSpacing())
    resampler.SetOutputDirection(image.GetDirection())
    resampler.SetOutputOrigin(image.GetOrigin())
    resampler.SetTransform(sitk.Transform())
    resampler.SetInterpolator(sitk.sitkNearestNeighbor)
    return resampler.Execute(segmentation_image)

In [17]:
from radiomics import featureextractor

features_dict = {}
x = []

for i in range(len(weight_folders)):
# for i in range(6):
    segmentation_path = seg_paths[i]
    segmentation_image = sitk.ReadImage(segmentation_path)
    segmentation = sitk.GetArrayFromImage(segmentation_image)

    reader = sitk.ImageSeriesReader()
    dicom_names = reader.GetGDCMSeriesFileNames(weight_folders[i])
    reader.SetFileNames(dicom_names)
    image = reader.Execute()

    patient_id = extract_patient_name(weight_folders[i])
    if patient_id is None:
        print(f"Warning: Could not extract patient ID from {weight_folders[i]}. Skipping.")
        continue
    
    patient_year = extract_patient_year(weight_folders[i])
    if patient_year is None:
        print(f"Warning: Could not extract patient year from {weight_folders[i]}. Skipping.")
        continue

    try:
        patient_year = int(patient_year[-4:])
    except ValueError:
        print(f"Warning: Invalid patient year format: {patient_year}. Skipping.")
        continue
    print(patient_id,patient_year)
    segmentation_resampled = resample_segmentation_to_image(segmentation_image, image)
    segmentation_resampled_array = sitk.GetArrayFromImage(segmentation_resampled)
    
    if np.sum(segmentation_resampled_array) == 0:
        print(f"Warning: The resampled segmentation mask for {segmentation_path} is empty (no segmented regions found).")
        continue

    extractor = featureextractor.RadiomicsFeatureExtractor()
    extractor.enableAllFeatures()

    features = extractor.execute(image, segmentation_resampled)
    
    x.append(f"{patient_id}_{patient_year}")

    # Store features in a list to handle repeated patient IDs
    key = f"{patient_id}_{patient_year}"
    if key in features_dict:
        suffix = 1
        while f"{key}_{suffix}" in features_dict:
            suffix += 1
        key = f"{key}_{suffix}"

    if key not in features_dict:
        features_dict[key] = []
    features_dict[key].append(features)

VS-SEG-003 1993


parameter force2D must be set to True to enable shape2D extraction
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


VS-SEG-003 1995


parameter force2D must be set to True to enable shape2D extraction
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


VS-SEG-003 1997


...

parameter force2D must be set to True to enable shape2D extraction
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


VS-MC-RC-101 1991


parameter force2D must be set to True to enable shape2D extraction
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


VS-MC-RC-102 1998


parameter force2D must be set to True to enable shape2D extraction
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


In [85]:
x

['VS-SEG-003_1993',
 'VS-SEG-003_1995',
 'VS-SEG-003_1997',
 'VS-SEG-010_1993',
 'VS-SEG-010_1996',
 'VS-SEG-010_1999',
 'VS-SEG-013_1991',
 'VS-SEG-013_1995',
 'VS-SEG-013_1999',
 'VS-SEG-021_1996',
 'VS-SEG-021_1999',
 'VS-SEG-025_1992',
 'VS-SEG-025_1996',
 'VS-SEG-025_1999',
 'VS-SEG-028_1990',
 'VS-SEG-028_1996',
 'VS-SEG-028_2000',
 'VS-SEG-030_1993',
 'VS-SEG-030_1993',
 'VS-SEG-030_1994',
 'VS-SEG-031_1989',
 'VS-SEG-031_1996',
 'VS-SEG-031_2000',
 'VS-SEG-032_1994',
 'VS-SEG-032_1996',
 'VS-SEG-032_2000',
 'VS-SEG-034_1996',
 'VS-SEG-034_1999',
 'VS-SEG-034_2000',
 'VS-SEG-041_1996',
 'VS-SEG-041_1999',
 'VS-SEG-042_1994',
 'VS-SEG-042_1997',
 'VS-SEG-042_1999',
 'VS-SEG-044_1992',
 'VS-SEG-044_1994',
 'VS-SEG-044_1995',
 'VS-SEG-045_1990',
 'VS-SEG-045_1995',
 'VS-SEG-045_2000',
 'VS-SEG-049_1995',
 'VS-SEG-049_1997',
 'VS-SEG-058_1994',
 'VS-SEG-058_1998',
 'VS-SEG-058_2001',
 'VS-SEG-059_1994',
 'VS-SEG-059_1997',
 'VS-SEG-059_2000',
 'VS-SEG-060_1994',
 'VS-SEG-060_1995',


In [86]:
voxel_volume_feature = {}
for key, feature_list in features_dict.items():
    for feature in feature_list:
        value = feature.get('original_shape_MeshVolume', None)
        voxel_volume_feature[key] = value.item() if isinstance(value, np.ndarray) else value
len(voxel_volume_feature)


301

In [87]:
voxel_volume_feature

{'VS-SEG-003_1993': 880.5233845949246,
 'VS-SEG-003_1995': 3145.3133775388014,
 'VS-SEG-003_1997': 2698.6396784584667,
 'VS-SEG-010_1993': 5117.444550127759,
 'VS-SEG-010_1996': 1629.0708055005846,
 'VS-SEG-010_1999': 802.4242824982265,
 'VS-SEG-013_1991': 73.84254329813322,
 'VS-SEG-013_1995': 333.16630394251973,
 'VS-SEG-013_1999': 529.7363281249998,
 'VS-SEG-021_1996': 2631.554796723139,
 'VS-SEG-021_1999': 2748.2222423354974,
 'VS-SEG-025_1992': 111.02028334221357,
 'VS-SEG-025_1996': 743.2763803142133,
 'VS-SEG-025_1999': 1613.2947487205859,
 'VS-SEG-028_1990': 255.67897188929012,
 'VS-SEG-028_1996': 464.7092111456221,
 'VS-SEG-028_2000': 489.45320423087975,
 'VS-SEG-030_1993': 2494.5436845548415,
 'VS-SEG-030_1993_1': 2677.548811157017,
 'VS-SEG-030_1994': 3828.012351525383,
 'VS-SEG-031_1989': 108.55041105746047,
 'VS-SEG-031_1996': 273.45443334715253,
 'VS-SEG-031_2000': 189.69610958735495,
 'VS-SEG-032_1994': 3263.522442803967,
 'VS-SEG-032_1996': 1868.5376306558076,
 'VS-SEG-

In [88]:
voxel_volume_feature['VS-MC-RC-012_1995'],voxel_volume_feature['VS-MC-RC-012_1995_1'] 

(14726.379495585357, 11946.920327381935)

In [89]:
import pandas as pd
import numpy as np

data = []
for key, value in voxel_volume_feature.items():
    parts = key.split('_')

    if len(parts) > 2:
        patient_id = parts[0]
        year = parts[1] + "_" + parts[2]  # Create year_1 format
    else:
        patient_id, year = parts[0], parts[1]

    data.append((patient_id, year, value))


df = pd.DataFrame(data, columns=['Patient_ID', 'Year', 'Volume'])

# Pivot the DataFrame to create a table with Patient_ID as the index, Year as columns, and Volume as the values
volume_table = df.pivot(index='Patient_ID', columns='Year', values='Volume')

# Replace missing values with "N/A"
volume_table = volume_table.fillna("NaN")

In [90]:
volume_table

Year,1986,1987,1988,1989,1990,1990_1,1991,1992,1993,1993_1,...,1995_1,1996,1996_1,1997,1997_1,1998,1998_1,1999,2000,2001
Patient_ID,,,,,,,,,,,,,,,,,,,,,
VS-MC-RC-001,NaN,NaN,193.155072,176.648758,142.324132,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
VS-MC-RC-002,NaN,240.964327,722.544553,1309.200949,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
VS-MC-RC-004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,480.145557,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
VS-MC-RC-005,NaN,NaN,1411.34254,NaN,NaN,NaN,3667.495958,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
VS-MC-RC-006,NaN,NaN,NaN,NaN,NaN,NaN,1255.555234,1312.419457,1460.938979,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VS-SEG-195,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,166.540492,NaN
VS-SEG-197,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1156.596678,NaN,NaN,NaN,2338.10842,NaN,NaN,NaN,NaN
VS-SEG-199,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2995.068412,NaN,4097.080922,NaN,NaN,NaN,NaN


In [ ]:
# dest_path = r'C:\Users\Acer\Desktop\3dVS\RFcsv\volume_data1.csv'
# volume_table.to_csv(dest_path)

In [92]:
import pandas as pd
import numpy as np

volume_diff_column = []


for idx, row in volume_table.iterrows():

    numeric_values = pd.to_numeric(row.dropna(), errors='coerce').dropna().values
    if len(numeric_values) > 1:
        first_numeric = int(numeric_values[0])
        last_numeric = int(numeric_values[-1])
        volume_diff = last_numeric - first_numeric
    else:
        volume_diff = np.nan


    volume_diff_column.append(volume_diff)

# Add the calculated 'volume_diff' as a new column
volume_table['volume_diff'] = volume_diff_column

# Create a new column based on whether the volume_diff is positive or negative
volume_table['change'] = volume_table['volume_diff'].apply(lambda x: 'increased' if x > 0 else 'not increased')


In [ ]:
# dest_path = r'C:\Users\Acer\Desktop\3dVS\RFcsv\volume_data2.csv'
# volume_table.to_csv(dest_path)

In [ ]:
feature_names= ['original_shape_Elongation', 'original_shape_Flatness', 'original_shape_LeastAxisLength', 'original_shape_MajorAxisLength', 'original_shape_Maximum2DDiameterColumn', 'original_shape_Maximum2DDiameterRow', 'original_shape_Maximum2DDiameterSlice', 'original_shape_Maximum3DDiameter', 'original_shape_MeshVolume', 'original_shape_MinorAxisLength', 'original_shape_Sphericity', 'original_shape_SurfaceArea', 'original_shape_SurfaceVolumeRatio', 'original_shape_VoxelVolume', 'original_firstorder_10Percentile', 'original_firstorder_90Percentile', 'original_firstorder_Energy', 'original_firstorder_Entropy', 'original_firstorder_InterquartileRange', 'original_firstorder_Kurtosis', 'original_firstorder_Maximum', 'original_firstorder_MeanAbsoluteDeviation', 'original_firstorder_Mean', 'original_firstorder_Median', 'original_firstorder_Minimum', 'original_firstorder_Range', 'original_firstorder_RobustMeanAbsoluteDeviation', 'original_firstorder_RootMeanSquared', 'original_firstorder_Skewness', 'original_firstorder_TotalEnergy', 'original_firstorder_Uniformity', 'original_firstorder_Variance', 'original_glcm_Autocorrelation', 'original_glcm_ClusterProminence', 'original_glcm_ClusterShade', 'original_glcm_ClusterTendency', 'original_glcm_Contrast', 'original_glcm_Correlation', 'original_glcm_DifferenceAverage', 'original_glcm_DifferenceEntropy', 'original_glcm_DifferenceVariance', 'original_glcm_Id', 'original_glcm_Idm', 'original_glcm_Idmn', 'original_glcm_Idn', 'original_glcm_Imc1', 'original_glcm_Imc2', 'original_glcm_InverseVariance', 'original_glcm_JointAverage', 'original_glcm_JointEnergy', 'original_glcm_JointEntropy', 'original_glcm_MCC', 'original_glcm_MaximumProbability', 'original_glcm_SumAverage', 'original_glcm_SumEntropy', 'original_glcm_SumSquares', 'original_gldm_DependenceEntropy', 'original_gldm_DependenceNonUniformity', 'original_gldm_DependenceNonUniformityNormalized', 'original_gldm_DependenceVariance', 'original_gldm_GrayLevelNonUniformity', 'original_gldm_GrayLevelVariance', 'original_gldm_HighGrayLevelEmphasis', 'original_gldm_LargeDependenceEmphasis', 'original_gldm_LargeDependenceHighGrayLevelEmphasis', 'original_gldm_LargeDependenceLowGrayLevelEmphasis', 'original_gldm_LowGrayLevelEmphasis', 'original_gldm_SmallDependenceEmphasis', 'original_gldm_SmallDependenceHighGrayLevelEmphasis', 'original_gldm_SmallDependenceLowGrayLevelEmphasis', 'original_glrlm_GrayLevelNonUniformity', 'original_glrlm_GrayLevelNonUniformityNormalized', 'original_glrlm_GrayLevelVariance', 'original_glrlm_HighGrayLevelRunEmphasis', 'original_glrlm_LongRunEmphasis', 'original_glrlm_LongRunHighGrayLevelEmphasis', 'original_glrlm_LongRunLowGrayLevelEmphasis', 'original_glrlm_LowGrayLevelRunEmphasis', 'original_glrlm_RunEntropy', 'original_glrlm_RunLengthNonUniformity', 'original_glrlm_RunLengthNonUniformityNormalized', 'original_glrlm_RunPercentage', 'original_glrlm_RunVariance', 'original_glrlm_ShortRunEmphasis', 'original_glrlm_ShortRunHighGrayLevelEmphasis', 'original_glrlm_ShortRunLowGrayLevelEmphasis', 'original_glszm_GrayLevelNonUniformity', 'original_glszm_GrayLevelNonUniformityNormalized', 'original_glszm_GrayLevelVariance', 'original_glszm_HighGrayLevelZoneEmphasis', 'original_glszm_LargeAreaEmphasis', 'original_glszm_LargeAreaHighGrayLevelEmphasis', 'original_glszm_LargeAreaLowGrayLevelEmphasis', 'original_glszm_LowGrayLevelZoneEmphasis', 'original_glszm_SizeZoneNonUniformity', 'original_glszm_SizeZoneNonUniformityNormalized', 'original_glszm_SmallAreaEmphasis', 'original_glszm_SmallAreaHighGrayLevelEmphasis', 'original_glszm_SmallAreaLowGrayLevelEmphasis', 'original_glszm_ZoneEntropy', 'original_glszm_ZonePercentage', 'original_glszm_ZoneVariance', 'original_ngtdm_Busyness', 'original_ngtdm_Coarseness', 'original_ngtdm_Complexity', 'original_ngtdm_Contrast', 'original_ngtdm_Strength']

107

In [ ]:
feature_names

['original_shape_Elongation',
 'original_shape_Flatness',
 'original_shape_LeastAxisLength',
 'original_shape_MajorAxisLength',
 'original_shape_Maximum2DDiameterColumn',
 'original_shape_Maximum2DDiameterRow',
 'original_shape_Maximum2DDiameterSlice',
 'original_shape_Maximum3DDiameter',
 'original_shape_MeshVolume',
 'original_shape_MinorAxisLength',
 'original_shape_Sphericity',
 'original_shape_SurfaceArea',
 'original_shape_SurfaceVolumeRatio',
 'original_shape_VoxelVolume',
 'original_firstorder_10Percentile',
 'original_firstorder_90Percentile',
 'original_firstorder_Energy',
 'original_firstorder_Entropy',
 'original_firstorder_InterquartileRange',
 'original_firstorder_Kurtosis',
 'original_firstorder_Maximum',
 'original_firstorder_MeanAbsoluteDeviation',
 'original_firstorder_Mean',
 'original_firstorder_Median',
 'original_firstorder_Minimum',
 'original_firstorder_Range',
 'original_firstorder_RobustMeanAbsoluteDeviation',
 'original_firstorder_RootMeanSquared',
 'original

In [95]:
for feature in feature_names:
    volume_table[feature] = float('nan')

C:\Users\Acer\AppData\Local\Temp\ipykernel_2900\3373166795.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  volume_table[feature] = float('nan')
C:\Users\Acer\AppData\Local\Temp\ipykernel_2900\3373166795.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  volume_table[feature] = float('nan')
C:\Users\Acer\AppData\Local\Temp\ipykernel_2900\3373166795.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all colum

In [96]:
for idx, row in volume_table.iterrows():
    numeric_values = pd.to_numeric(row.dropna(), errors='coerce').dropna()
    if numeric_values.empty:
        continue

    year = min(numeric_values.index.tolist())  # earliest available year
    dict_key = f"{idx}_{year}"

    # Check if the key exists in features_dict
    if dict_key in features_dict:
        feature_values = features_dict[dict_key][0]
        for feature in feature_names:
            volume_table.at[idx, feature] = feature_values[feature]
    else:
        print(f"Warning: {dict_key} not found in features_dict.")

In [97]:
volume_table

Year,1986,1987,1988,1989,1990,1990_1,1991,1992,1993,1993_1,...,original_glszm_SmallAreaHighGrayLevelEmphasis,original_glszm_SmallAreaLowGrayLevelEmphasis,original_glszm_ZoneEntropy,original_glszm_ZonePercentage,original_glszm_ZoneVariance,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength
Patient_ID,,,,,,,,,,,,,,,,,,,,,
VS-MC-RC-001,NaN,NaN,193.155072,176.648758,142.324132,NaN,NaN,NaN,NaN,NaN,...,10.695441,0.076159,4.020056,0.045500,8.521760e+03,14.322749,0.005272,16.255589,0.019169,0.263056
VS-MC-RC-002,NaN,240.964327,722.544553,1309.200949,NaN,NaN,NaN,NaN,NaN,NaN,...,480.178708,0.004672,6.442091,0.612416,1.898097e+00,0.149696,0.005885,6488.583068,0.304126,7.944378
VS-MC-RC-004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,32.093094,0.029082,7.190271,0.010443,3.126897e+06,61.614317,0.000073,62.767139,0.027590,0.011189
VS-MC-RC-005,NaN,NaN,1411.34254,NaN,NaN,NaN,3667.495958,NaN,NaN,NaN,...,1403.268164,0.000975,7.284012,0.535631,4.696216e+00,0.178940,0.000656,57029.297654,0.081517,12.098159
VS-MC-RC-006,NaN,NaN,NaN,NaN,NaN,NaN,1255.555234,1312.419457,1460.938979,NaN,...,35.924964,0.077624,5.696608,0.077204,1.368253e+04,7.544970,0.000963,164.020212,0.028489,0.178225
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VS-SEG-195,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2591.392859,0.001952,6.126663,0.862069,2.944000e-01,0.013151,0.023994,27866.168951,2.382383,50.594926
VS-SEG-197,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,112.478485,0.010389,6.023579,0.258682,3.014484e+02,0.474109,0.004280,536.160015,0.123540,0.938516
VS-SEG-199,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,17.464088,0.078385,5.835668,0.034917,2.251998e+05,88.305997,0.000146,179.378541,0.017395,0.053233


In [1]:
# dest_path = r'C:\Users\Acer\Desktop\3dVS\RFcsv\volume_data3.csv'
# volume_table.to_csv(dest_path)

In [99]:
cols_to_drop = [col for col in volume_table.columns if str(col).strip().startswith(tuple("0123456789"))]
# cols_to_drop

In [100]:
volume_table = volume_table.drop(columns=cols_to_drop)

In [101]:
volume_table

Year,volume_diff,change,original_shape_Elongation,original_shape_Flatness,original_shape_LeastAxisLength,original_shape_MajorAxisLength,original_shape_Maximum2DDiameterColumn,original_shape_Maximum2DDiameterRow,original_shape_Maximum2DDiameterSlice,original_shape_Maximum3DDiameter,...,original_glszm_SmallAreaHighGrayLevelEmphasis,original_glszm_SmallAreaLowGrayLevelEmphasis,original_glszm_ZoneEntropy,original_glszm_ZonePercentage,original_glszm_ZoneVariance,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength
Patient_ID,,,,,,,,,,,,,,,,,,,,,
VS-MC-RC-001,-51.0,not increased,0.574892,0.420573,4.667643,11.098302,7.692140,10.667506,9.907894,11.824483,...,10.695441,0.076159,4.020056,0.045500,8.521760e+03,14.322749,0.005272,16.255589,0.019169,0.263056
VS-MC-RC-002,1069.0,increased,0.574886,0.383779,4.624665,12.050322,14.526802,8.669793,12.282286,14.842891,...,480.178708,0.004672,6.442091,0.612416,1.898097e+00,0.149696,0.005885,6488.583068,0.304126,7.944378
VS-MC-RC-004,-13391.0,not increased,0.767449,0.750977,24.363244,32.442076,37.514031,33.636311,35.544042,38.487053,...,32.093094,0.029082,7.190271,0.010443,3.126897e+06,61.614317,0.000073,62.767139,0.027590,0.011189
VS-MC-RC-005,2256.0,increased,0.785051,0.676358,10.682136,15.793601,19.495446,15.119270,20.937774,21.244994,...,1403.268164,0.000975,7.284012,0.535631,4.696216e+00,0.178940,0.000656,57029.297654,0.081517,12.098159
VS-MC-RC-006,205.0,increased,0.560217,0.390187,8.214488,21.052694,23.263138,14.840822,24.317952,24.568533,...,35.924964,0.077624,5.696608,0.077204,1.368253e+04,7.544970,0.000963,164.020212,0.028489,0.178225
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VS-SEG-195,NaN,not increased,0.473118,0.406433,4.480535,11.024049,11.574484,6.088697,10.146551,11.833204,...,2591.392859,0.001952,6.126663,0.862069,2.944000e-01,0.013151,0.023994,27866.168951,2.382383,50.594926
VS-SEG-197,1251.0,increased,0.763451,0.612028,9.262721,15.134468,17.845172,13.550974,17.346565,19.078826,...,112.478485,0.010389,6.023579,0.258682,3.014484e+02,0.474109,0.004280,536.160015,0.123540,0.938516
VS-SEG-199,1102.0,increased,0.880602,0.623155,12.667649,20.328248,25.398081,20.264196,27.537262,30.923181,...,17.464088,0.078385,5.835668,0.034917,2.251998e+05,88.305997,0.000146,179.378541,0.017395,0.053233


In [ ]:
# dest_path = r'C:\Users\Acer\Desktop\3dVS\RFcsv\volume_data4.csv'
# volume_table.to_csv(dest_path)

In [103]:
import numpy as np
np.unique(volume_table['change'])

array(['increased', 'not increased'], dtype=object)

In [104]:
volume_table['change'] = volume_table['change'].map({
    'increased': 1,
    'not increased': 0
})

In [105]:
volume_table

Year,volume_diff,change,original_shape_Elongation,original_shape_Flatness,original_shape_LeastAxisLength,original_shape_MajorAxisLength,original_shape_Maximum2DDiameterColumn,original_shape_Maximum2DDiameterRow,original_shape_Maximum2DDiameterSlice,original_shape_Maximum3DDiameter,...,original_glszm_SmallAreaHighGrayLevelEmphasis,original_glszm_SmallAreaLowGrayLevelEmphasis,original_glszm_ZoneEntropy,original_glszm_ZonePercentage,original_glszm_ZoneVariance,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength
Patient_ID,,,,,,,,,,,,,,,,,,,,,
VS-MC-RC-001,-51.0,0,0.574892,0.420573,4.667643,11.098302,7.692140,10.667506,9.907894,11.824483,...,10.695441,0.076159,4.020056,0.045500,8.521760e+03,14.322749,0.005272,16.255589,0.019169,0.263056
VS-MC-RC-002,1069.0,1,0.574886,0.383779,4.624665,12.050322,14.526802,8.669793,12.282286,14.842891,...,480.178708,0.004672,6.442091,0.612416,1.898097e+00,0.149696,0.005885,6488.583068,0.304126,7.944378
VS-MC-RC-004,-13391.0,0,0.767449,0.750977,24.363244,32.442076,37.514031,33.636311,35.544042,38.487053,...,32.093094,0.029082,7.190271,0.010443,3.126897e+06,61.614317,0.000073,62.767139,0.027590,0.011189
VS-MC-RC-005,2256.0,1,0.785051,0.676358,10.682136,15.793601,19.495446,15.119270,20.937774,21.244994,...,1403.268164,0.000975,7.284012,0.535631,4.696216e+00,0.178940,0.000656,57029.297654,0.081517,12.098159
VS-MC-RC-006,205.0,1,0.560217,0.390187,8.214488,21.052694,23.263138,14.840822,24.317952,24.568533,...,35.924964,0.077624,5.696608,0.077204,1.368253e+04,7.544970,0.000963,164.020212,0.028489,0.178225
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VS-SEG-195,NaN,0,0.473118,0.406433,4.480535,11.024049,11.574484,6.088697,10.146551,11.833204,...,2591.392859,0.001952,6.126663,0.862069,2.944000e-01,0.013151,0.023994,27866.168951,2.382383,50.594926
VS-SEG-197,1251.0,1,0.763451,0.612028,9.262721,15.134468,17.845172,13.550974,17.346565,19.078826,...,112.478485,0.010389,6.023579,0.258682,3.014484e+02,0.474109,0.004280,536.160015,0.123540,0.938516
VS-SEG-199,1102.0,1,0.880602,0.623155,12.667649,20.328248,25.398081,20.264196,27.537262,30.923181,...,17.464088,0.078385,5.835668,0.034917,2.251998e+05,88.305997,0.000146,179.378541,0.017395,0.053233


In [ ]:
dest_path = r'/home/omen/Documents/Nafisha/VS/VS_refactored_code/RandomForest/CSVs/RFcsv.csv'
volume_table.to_csv(dest_path)

In [5]:
volume_table

,volume_diff,change,original_shape_Elongation,original_shape_Flatness,original_shape_LeastAxisLength,original_shape_MajorAxisLength,original_shape_Maximum2DDiameterColumn,original_shape_Maximum2DDiameterRow,original_shape_Maximum2DDiameterSlice,original_shape_Maximum3DDiameter,...,original_glszm_SmallAreaHighGrayLevelEmphasis,original_glszm_SmallAreaLowGrayLevelEmphasis,original_glszm_ZoneEntropy,original_glszm_ZonePercentage,original_glszm_ZoneVariance,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength
Patient_ID,,,,,,,,,,,,,,,,,,,,,
VS-MC-RC-001,-51.0,0,0.574892,0.420573,4.667643,11.098302,7.692140,10.667506,9.907894,11.824483,...,10.695441,0.076159,4.020056,0.045500,8.521760e+03,14.322749,0.005272,16.255589,0.019169,0.263056
VS-MC-RC-002,1069.0,1,0.574886,0.383779,4.624665,12.050322,14.526802,8.669793,12.282286,14.842891,...,480.178708,0.004672,6.442091,0.612416,1.898097e+00,0.149696,0.005885,6488.583068,0.304126,7.944378
VS-MC-RC-004,-13391.0,0,0.767449,0.750977,24.363244,32.442076,37.514031,33.636311,35.544042,38.487053,...,32.093094,0.029082,7.190271,0.010443,3.126897e+06,61.614317,0.000073,62.767139,0.027590,0.011189
VS-MC-RC-005,2256.0,1,0.785051,0.676358,10.682136,15.793601,19.495446,15.119270,20.937774,21.244994,...,1403.268164,0.000975,7.284012,0.535631,4.696216e+00,0.178940,0.000656,57029.297654,0.081517,12.098159
VS-MC-RC-006,205.0,1,0.560217,0.390187,8.214488,21.052694,23.263138,14.840822,24.317952,24.568533,...,35.924964,0.077624,5.696608,0.077204,1.368253e+04,7.544970,0.000963,164.020212,0.028489,0.178225
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VS-SEG-195,NaN,0,0.473118,0.406433,4.480535,11.024049,11.574484,6.088697,10.146551,11.833204,...,2591.392859,0.001952,6.126663,0.862069,2.944000e-01,0.013151,0.023994,27866.168951,2.382383,50.594926
VS-SEG-197,1251.0,1,0.763451,0.612028,9.262721,15.134468,17.845172,13.550974,17.346565,19.078826,...,112.478485,0.010389,6.023579,0.258682,3.014484e+02,0.474109,0.004280,536.160015,0.123540,0.938516
VS-SEG-199,1102.0,1,0.880602,0.623155,12.667649,20.328248,25.398081,20.264196,27.537262,30.923181,...,17.464088,0.078385,5.835668,0.034917,2.251998e+05,88.305997,0.000146,179.378541,0.017395,0.053233


In [ ]:
selected_feature_names_rfe = ['volume_diff', 'change', 'original_shape_MajorAxisLength', 'original_shape_Maximum2DDiameterRow',
       'original_shape_Maximum3DDiameter', 'original_glcm_Imc1',
       'original_glcm_MCC', 'original_gldm_DependenceNonUniformity',
       'original_glrlm_GrayLevelNonUniformity',
       'original_glrlm_RunLengthNonUniformity',
       'original_glszm_LargeAreaHighGrayLevelEmphasis',
       'original_ngtdm_Coarseness']
filtered_df = volume_table[selected_feature_names_rfe]
# len(selected_feature_names_rfe)

In [7]:
filtered_df

,volume_diff,change,original_shape_MajorAxisLength,original_shape_Maximum2DDiameterRow,original_shape_Maximum3DDiameter,original_glcm_Imc1,original_glcm_MCC,original_gldm_DependenceNonUniformity,original_glrlm_GrayLevelNonUniformity,original_glrlm_RunLengthNonUniformity,original_glszm_LargeAreaHighGrayLevelEmphasis,original_ngtdm_Coarseness
Patient_ID,,,,,,,,,,,,
VS-MC-RC-001,-51.0,0,11.098302,10.667506,11.824483,-0.055402,0.324320,53.124629,222.335594,281.082912,1.741270e+04,0.005272
VS-MC-RC-002,1069.0,1,12.050322,8.669793,14.842891,-0.147962,0.454266,380.436242,41.646779,1070.016494,2.363640e+03,0.005885
VS-MC-RC-004,-13391.0,0,32.442076,33.636311,38.487053,-0.402398,0.941602,16772.235332,18344.919598,94705.574083,3.764821e+08,0.000073
VS-MC-RC-005,2256.0,1,15.793601,15.119270,21.244994,-0.084973,0.817496,3718.285922,397.743860,11970.271692,1.321722e+04,0.000656
VS-MC-RC-006,205.0,1,21.052694,14.840822,24.568533,-0.100471,0.581540,815.913548,976.411879,4895.986528,5.086712e+05,0.000963
...,...,...,...,...,...,...,...,...,...,...,...,...
VS-SEG-195,NaN,0,11.024049,6.088697,11.833204,-0.665496,0.974734,108.379310,3.299489,167.773191,5.449440e+03,0.023994
VS-SEG-197,1251.0,1,15.134468,13.550974,19.078826,-0.083603,0.490837,296.263580,136.934265,1598.800575,1.087350e+05,0.004280
VS-SEG-199,1102.0,1,20.328248,20.264196,30.923181,-0.092100,0.519013,4153.128225,6676.043156,23640.329189,2.321828e+06,0.000146


In [ ]:
dest_path = r'/home/omen/Documents/Nafisha/VS/VS_refactored_code/RandomForest/CSVs/RFselected.csv'
filtered_df.to_csv(dest_path)